In [1]:
import pandas as pd
import numpy as np
from bson import ObjectId

import keyring
import mysql.connector

import arcgis
from arcgis.gis import GIS
import datetime as dt
import getpass
from IPython.display import HTML

from arcgis.geocoding import geocode
from arcgis.features import Feature, FeatureSet, FeatureLayerCollection, use_proximity, FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.network.analysis import generate_origin_destination_cost_matrix

In [2]:
username = "Glodanale"
password = keyring.get_password("arcgis", "Glodanale")
gis = GIS("https://www.arcgis.com", "Glodanale", password)

In [3]:
def pullEIDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM ei"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "ei"
    
    cursor.close()
    connection.close()
    
    return df

In [4]:
def pullChildDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM child"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "child"
    
    cursor.close()
    connection.close()
    
    return df

In [5]:
dataEI = pullEIDataFromMySQL()
dataChild = pullChildDataFromMySQL()

In [6]:
print(dataEI)
print("-------------------------------------------------------------------------------------------------")
print(dataChild)

                         _id  FirstName    LastName  ServiceHours  \
0   65d75a2ce09e67e025e8cd30       Mary  Richardson           0.0   
1   65d75bb0e09e67e025e8cd31    Kirstin      Tipton           1.0   
2   65d75c59e09e67e025e8cd32      Kassi      Knight           0.0   
3   65d75c87e09e67e025e8cd33     Rachel      Taylor           0.0   
4   65d75cb9e09e67e025e8cd34    Brannan     Vanover           0.0   
5   65d75ce1e09e67e025e8cd35     Briana     Ailshie           0.0   
6   65d75d08e09e67e025e8cd36       Dawn     Martino           0.0   
7   65d75d4ce09e67e025e8cd37      Casey      Crumly           0.0   
8   65d75d80e09e67e025e8cd38        Kim      Risner           0.0   
9   65d75da9e09e67e025e8cd39      Lydia        Ugwu           0.0   
10  65d75dd5e09e67e025e8cd3a      Emily    Jennings           0.0   
11  65d75dfae09e67e025e8cd3b      Faith       Ihrig           0.0   
12  65d75e28e09e67e025e8cd3c  Stephanie       Cowan           0.0   

                     Address1    

In [7]:
allData = pd.concat([dataEI, dataChild], ignore_index=True)
print(allData)

                          _id FirstName    LastName  ServiceHours  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson           0.0   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton           1.0   
2    65d75c59e09e67e025e8cd32     Kassi      Knight           0.0   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor           0.0   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover           0.0   
..                        ...       ...         ...           ...   
204  65e6038938f826f29267202f     Judah     Overbay           1.0   
205  65e603f238f826f292672030    Bianca       Pogue           1.0   
206  65e6044938f826f292672031   Emorett     Woodall           1.0   
207  65e6049b38f826f292672032      Remi       Woody           0.5   
208  65ef076606e0a866af56d202  Madilynn      Phipps           1.0   

                      Address1           City State Zipcode      County  \
0     904 West Hillcrest Drive   Johnson City    TN   37604  Washington   
1        151 Valley V

In [8]:
def combine_address(row):
    return f"{row['Address1']}, {row['City']}, {row['State']} {row['Zipcode']}"

allData["Full_Address"] = allData.apply(combine_address, axis=1)
#print(allData)

featureSetDF = allData.copy()
featureSetDF = allData.drop(columns=["ServiceHours", "Address1", "City", "State", "Zipcode", "County"])
featureSetDF = featureSetDF[["_id","FirstName", "LastName", "Full_Address", "Longitude", "Latitude", "Role"]]
print(featureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
204  65e6038938f826f29267202f     Judah     Overbay   
205  65e603f238f826f292672030    Bianca       Pogue   
206  65e6044938f826f292672031   Emorett     Woodall   
207  65e6049b38f826f292672032      Remi       Woody   
208  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [9]:
duplicates = featureSetDF[featureSetDF.duplicated(subset=["Longitude", "Latitude"], keep=False)]
print(duplicates)

                          _id     FirstName   LastName  \
40   65d77fdce09e67e025e8cd62         Dixie      Jones   
41   65d78068e09e67e025e8cd63        Gracie      Jones   
42   65d780cee09e67e025e8cd64      Benjamin   Keasling   
43   65d78173e09e67e025e8cd65         Henry   Keasling   
48   65d7842ee09e67e025e8cd6a        Hunter     Morton   
49   65d78493e09e67e025e8cd6b          Luke     Morton   
55   65d787f5e09e67e025e8cd71       Emersyn      Jones   
56   65d7886ae09e67e025e8cd72        Olivia      Jones   
65   65d8b679e09e67e025e8cd7b        Larson     Ketron   
66   65d8b6f8e09e67e025e8cd7c         Mason     Ketron   
67   65d8b770e09e67e025e8cd7d   Christopher      Kimes   
68   65d8b7d1e09e67e025e8cd7e        Corbin      Kimes   
69   65d8b845e09e67e025e8cd7f          Aria  McClellan   
70   65d8b8b7e09e67e025e8cd80        Aurora  McClellan   
72   65d8b9c0e09e67e025e8cd82       Charlie    Mullins   
76   65d8bc1ee09e67e025e8cd86        Armani        Arp   
80   65d8be2ae

In [10]:
uniqueFeatureSetDF = featureSetDF.copy()
uniqueFeatureSetDF = uniqueFeatureSetDF.drop_duplicates(subset=["Longitude", "Latitude"], keep="first")
uniqueFeatureSetDF = uniqueFeatureSetDF.reset_index(drop=True)
print(uniqueFeatureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
184  65e6038938f826f29267202f     Judah     Overbay   
185  65e603f238f826f292672030    Bianca       Pogue   
186  65e6044938f826f292672031   Emorett     Woodall   
187  65e6049b38f826f292672032      Remi       Woody   
188  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [11]:
# Define the number of rows per chunk
chunk_size = 10

# Get the total number of rows in the original dataframe
total_rows = len(uniqueFeatureSetDF)

# Initialize an empty list to store the chunks
chunks = []

# Iterate over the rows of the original dataframe
for i in range(0, total_rows, chunk_size):
    # Get a chunk of data with the specified chunk size
    chunk = uniqueFeatureSetDF.iloc[i:i+chunk_size]
    chunk.reset_index(drop=True, inplace=True)
    # Append the chunk to the list of chunks
    chunks.append(chunk)

# Print the number of chunks created
print("Number of chunks:", len(chunks))

# Print the first few rows of each chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print()


Number of chunks: 19
Chunk 1:
                        _id FirstName    LastName  \
0  65d75a2ce09e67e025e8cd30      Mary  Richardson   
1  65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2  65d75c59e09e67e025e8cd32     Kassi      Knight   
3  65d75c87e09e67e025e8cd33    Rachel      Taylor   
4  65d75cb9e09e67e025e8cd34   Brannan     Vanover   
5  65d75ce1e09e67e025e8cd35    Briana     Ailshie   
6  65d75d08e09e67e025e8cd36      Dawn     Martino   
7  65d75d4ce09e67e025e8cd37     Casey      Crumly   
8  65d75d80e09e67e025e8cd38       Kim      Risner   
9  65d75da9e09e67e025e8cd39     Lydia        Ugwu   

                                       Full_Address  Longitude   Latitude Role  
0  904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   ei  
1         151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   ei  
2      7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   ei  
3          502 Nunley Drive, Johnson City, TN 37604 -82.415306

In [12]:
def buildFeatureSet(df):
    features = []

    for index, row in df.iterrows():
        attributes = {"Address": row['Full_Address'], "Status": "unconnected", "OBJECTID": row['_id'], "Role": row['Role']}
        geometry = {"x": row["Longitude"], 'y': row["Latitude"]}
        feature = {"attributes": attributes, "geometry": geometry}
        features.append(feature)
        
    location_json = {
        "features": features,
        "spatialReference": {"wkid": 4326, "lastestWkid": 4326},
        "geometryType": "esriGeometryPoint",
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeString", "alias": "OBJECTID", "length": "50"},
            {"name": "Address", "type": "esriFieldTypeString", "alias": "Address", "length": "100"},
            {"name": "Status", "type": "esriFieldTypeString", "alias": "Status", "length": "50"},
            {"name": "Role", "type": "esriFieldTypeString", "alias": "Role", "length": "50"}
        ]
    }
        
    feature_set = FeatureSet.from_dict(location_json)
    return feature_set

In [13]:
def createFeatureLayer(feature_set, name):
    feature_layer_item = gis.content.import_data(feature_set, title=f"Feature Layer {name}")
    feature_layer_url = feature_layer_item.url
    #print("Feature Layer URL:", feature_layer_url)
    
    return feature_layer_item

In [14]:
def analyzeFeatureLayer(feature_layer_item):
    feature_layer = feature_layer_item.layers[0]
    features = feature_layer.query()

    for feature in features:
        print(feature)
        print("\n")

In [15]:
def process(df, name):
    feature_set = buildFeatureSet(df)
    feature_layer_item = createFeatureLayer(feature_set, name)
    analyzeFeatureLayer(feature_layer_item)
    
    return feature_layer_item

In [16]:
def updateDestinations(feature_layer_item, id_value):
    feature_layer = feature_layer_item.layers[0]
    feature_layer_query = feature_layer.query(where=f"OBJECTID = '{id_value}'")
    for feature in feature_layer_query.features:
        feature.attributes["Status"] = "connected"
    feature_layer.edit_features(updates=feature_layer_query.features)

In [17]:
def buildMatrix(origin, destination, name):
    items = gis.content.search(query="OD Cost Matrix", item_type="Feature Service")
    for item in items:
        item.delete()

    result = use_proximity.connect_origins_to_destinations(origins_layer=origin,
                                                            destinations_layer=destination,
                                                            origins_layer_route_id_field= "Status",
                                                            destinations_layer_route_id_field="Status",
                                                            output_name=f"OD Cost Matrix {name}",
                                                            gis=gis)

    print("I completed the function")

    od_cost_matrix_sublayer = FeatureLayer.fromitem(result, layer_id=0)
    print(type(od_cost_matrix_sublayer))
    print(od_cost_matrix_sublayer)
    od_df = od_cost_matrix_sublayer.query(where='1=1', as_df=True)
    print(od_df)

    return od_df

In [38]:
def child_child_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE child_id_1 = '{layerID}' and child_id_2 = '{loopID}' "
             f"OR child_id_1 = '{loopID}' and child_id_2 = '{layerID}'" )     

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {0.0}, driving_time = {0.0} " 
                       f"WHERE child_id_1 = '{layerID}' and child_id_2 = '{loopID}' "
                       f"OR child_id_1 = '{loopID}' and child_id_2 = '{layerID}'" )

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            layerID, loopID, None, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [39]:
def child_ei_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE child_id_1 = '{layerID}' and ei_id_1 = '{loopID}'")

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {0.0}, driving_time = {0.0} " 
                       f"WHERE child_id_1 = '{layerID}' and ei_id_1 = '{loopID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            layerID, None, loopID, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [40]:
def ei_child_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE ei_id_1 = '{layerID}' and child_id_1 = '{loopID}'")
    print(query)

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {0.0}, driving_time = {0.0} " 
                       f"WHERE ei_id_1 = '{layerID}' and child_id_1 = '{loopID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            loopID, None, layerID, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [41]:
def ei_ei_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE ei_id_1 = '{layerID}' and ei_id_2 = '{loopID}' "
             f"OR ei_id_1 = '{loopID}' and ei_id_2 = '{layerID}'")

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {0.0}, driving_time = {0.0} " 
                       f"WHERE ei_id_1 = '{layerID}' and ei_id_2 = '{loopID}' "
                       f"OR ei_id_1 = '{loopID}' and ei_id_2 = '{layerID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")

    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            None, None, layerID, loopID, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [44]:
def pushLocationDataToMySQL(locationDF, duplicates):
    if locationDF.empty:
        print("Error: locationDF is empty")
    else:
        connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
        )
    
        cursor = connection.cursor()
    
        print(f"\n\nDatabase connected\n\n")
    
        for index, row in locationDF.iterrows():
            print()
            if row.isnull().any():
                print(f"{row['From_OBJECTID']} to {row['To_OBJECTID']} contains null values")
            else:
                fromRole = row["From_Role"]
                toRole = row["To_Role"]
                miles = row['Total_Miles']
                minutes = row['Total_Minutes']
                fromID = row["From_OBJECTID"]
                toID = row["To_OBJECTID"]
                
                print(f"fromRole: {fromRole}   toRole:{toRole}")
                
                #make sure duplicates are entered in locationdata with 0 distance and 0 drive time
                if fromID in duplicates["_id"].values:
                    print("duplicate in fromID found")
                    from_filtered_ids = duplicates.loc[duplicates["Full_Address"] == row["From_Address"], "_id"].tolist()
                    for a in range(0, len(from_filtered_ids)):
                        loopID = from_filtered_ids[a]
                        
                        if fromID != loopID:
                            row1 = duplicates.loc[duplicates["_id"] == loopID, "Role"]
                            role1 = row1.iloc[0]
                            if fromRole == "child":
                                if role1 == "child":
                                    child_child_duplicates(fromID, loopID, cursor)
                                elif role1 == "ei":
                                    child_ei_duplicates(fromID, loopID, cursor)
                                else:
                                    print("Error with types")
                            elif fromRole == "ei":
                                if role1 == "child":
                                    ei_child_duplicates(fromID, loopID, cursor)
                                elif role1 == "ei":
                                    ei_ei_duplicates(fromID, loopID, cursor)
                                else:
                                    print("Error with types")
                            else:
                                print("Error finding role")
                        else:
                            print(f"IDs match: fromID={fromID}   loopID={loopID}")
                    #exit from_filtered_ids loop
                else:
                    from_filtered_ids = [row['From_OBJECTID']]
                         
                if toID in duplicates["_id"].values:
                    print("duplicate in toID found")
                    to_filtered_ids = duplicates.loc[duplicates["Full_Address"] == row["To_Address"], "_id"].tolist()
                    
                    for d in range(0, len(to_filtered_ids)):
                        loopID = to_filtered_ids[d]
                        if toID != loopID:
                            row1 = duplicates.loc[duplicates["_id"] == loopID, "Role"]
                            role1 = row1.iloc[0]
                            if fromRole == "child":
                                if role1 == "child":
                                    child_child_entry(toID, loopID, cursor)
                                elif role1 == "ei":
                                    child_ei_entry(toID, loopID, cursor)
                                else:
                                    print("Error with types")
                            elif fromRole == "ei":
                                if role1 == "child":
                                    ei_child_entry(toID, loopID, cursor)
                                elif role1 == "ei":
                                    ei_ei_entry(toID, loopID, cursor)
                                else:
                                    print("Error with types")
                            else:
                                print("Error finding role")
                        else:
                            print(f"IDs match: loopID={loopID}   toID={toID}")
                else:
                    to_filtered_ids = [row['To_OBJECTID']]
                #end of duplicate checking    
                
                #match all from_locations with all to_locations
                for i in range(0, len(from_filtered_ids)):
                    for j in range(0, len(to_filtered_ids)):
                        fromID = from_filtered_ids[i]
                        toID = to_filtered_ids[j]
                    
                        if fromRole == "child":
                            if toRole == "child":
                                child_child_entry(fromID, toID, cursor, miles=miles, minutes=minutes)
                            elif toRole == "ei":
                                child_ei_entry(fromID, toID, cursor, miles=miles, minutes=minutes)
                            else:
                                print("Error recieving to_role")
                        elif fromRole == "ei":
                            if toRole == "child":
                                ei_child_entry(fromID, toID, cursor, miles=miles, minutes=minutes)                                    
                            elif toRole == "ei":
                                ei_ei_entry(fromID, toID, cursor, miles=miles, minutes=minutes)                                
                            else:
                                print("Error recieving to_role")
                        else:
                            print("Error recieving from_role")
                #end of all matches
 
        connection.commit()
        cursor.close()
        connection.close()
        
        print("---------------------------------------------------------------------------------------------------------------------------------")
        print("All data entered into database")

In [23]:
def buildData(df, uniqueFeatureSetDF, duplicates): 
    for index, row in df.iterrows():        
        row_df = pd.DataFrame(row).transpose()
        row_df.reset_index(drop=True, inplace=True)
        if len(row_df) != 1:
            print("Error in df length")
        else:
            id_value = row_df.loc[0, "_id"]
            uniqueFeatureSetDF.drop(uniqueFeatureSetDF[uniqueFeatureSetDF['_id'] == id_value])
            destination = process(uniqueFeatureSetDF, 'destination')
            originLayer = process(row_df, id_value)
            originDestinationDF = buildMatrix(originLayer, destination, id_value)
            pushLocationDataToMySQL(originDestinationDF, duplicates)

In [24]:
#destination = process(uniqueFeatureSetDF, "destination")
#buildData(chunks[0], destination, duplicates)

In [26]:
uniqueFeatureSetDFCopy = uniqueFeatureSetDF.copy()
for index, row in chunks[0].iterrows():
    row_df = pd.DataFrame(row).transpose()
    row_df.reset_index(drop=True, inplace=True)
    print(row_df)
    id_value = row_df.loc[0, "_id"]
    print(id_value)
    newUniqueFeatureSetDF = uniqueFeatureSetDFCopy[uniqueFeatureSetDFCopy['_id'] != id_value]
    destinationLayer = process(newUniqueFeatureSetDF, "destination")
    originLayer = process(row_df, id_value)
    originDestinationDF = buildMatrix(originLayer, destinationLayer, id_value)
    print(originDestinationDF)
    break

                        _id FirstName    LastName  \
0  65d75a2ce09e67e025e8cd30      Mary  Richardson   

                                       Full_Address  Longitude   Latitude Role  
0  904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   ei  
65d75a2ce09e67e025e8cd30
{"geometry": {"x": -9125730.959520198, "y": 4400444.975741492, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "151 Valley View Drive, Abingdon, VA 24210", "Status": "unconnected", "OBJECTID": "65d75bb0e09e67e025e8cd31", "Role": "ei"}}


{"geometry": {"x": -9233350.116959408, "y": 4327769.409124248, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "7040 Blue Springs Parkway, Mosheim, TN 37818", "Status": "unconnected", "OBJECTID": "65d75c59e09e67e025e8cd32", "Role": "ei"}}


{"geometry": {"x": -9174429.90765625, "y": 4342721.095604265, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes"

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").
{"cost": 0.9400000000000001}


I completed the function
<class 'arcgis.features.layer.FeatureLayer'>
<FeatureLayer url:"https://services5.arcgis.com/ZYK688A64hFRZ5yX/arcgis/rest/services/OD_Cost_Matrix_65d75a2ce09e67e025e8cd30/FeatureServer/0">
     OBJECTID RouteName StartTime EndTime StartTimeUTC EndTimeUTC  \
0           1   1 - 100       NaT     NaT          NaT        NaT   
1           2   1 - 113       NaT     NaT          NaT        NaT   
2           3    1 - 10       NaT     NaT          NaT        NaT   
3           4    1 - 72       NaT     NaT          NaT        NaT   
4           5   1 - 126       NaT     NaT          NaT        NaT   
..        ...       ...       ...     ...          ...        ...   
183       184    1 - 88       NaT     NaT          NaT        NaT   
184       185    1 - 89       NaT     NaT          NaT        NaT   
185       186     1 - 9       NaT     NaT          NaT        NaT   
186       187    1 - 98       NaT     NaT          NaT        NaT   
187       188    1 - 99    

In [47]:
myODMatrix = originDestinationDF[['From_OBJECTID','From_Address','From_Role','To_OBJECTID','To_Address','To_Role','Total_Miles','Total_Minutes']]
pushLocationDataToMySQL(myODMatrix, duplicates)



Database connected



fromRole: ei   toRole:child
SELECT * FROM locationdata WHERE ei_id_1 = '65d75a2ce09e67e025e8cd30' and child_id_1 = '65d8d05fe09e67e025e8cda9'


DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

In [27]:
print(originDestinationDF)

   OBJECTID_1                                           Address ORIG_Status  \
0           1  904 West Hillcrest Drive, Johnson City, TN 37604   connected   

                   OBJECTID Role                   Status  \
0  65d75a2ce09e67e025e8cd30   ei  Same location as origin   

                                               SHAPE  
0  {"x": -9169621.1693, "y": 4344671.493299998, "...  


In [26]:
originDestinationDF.to_csv('file.csv', index=False)

In [46]:
print(od_df)

   OBJECTID RouteName StartTime EndTime StartTimeUTC EndTimeUTC  Total_Miles  \
0         1     1 - 1       NaT     NaT          NaT        NaT     3.290089   
1         2     1 - 2       NaT     NaT          NaT        NaT    13.811811   
2         3     1 - 3       NaT     NaT          NaT        NaT    19.906688   

   Total_Kilometers  Total_Minutes  TotalWait_Minutes  ...  RouteLayerItemURL  \
0          5.294896       7.917875               <NA>  ...               <NA>   
1            22.228      23.989688               <NA>  ...               <NA>   
2         32.036772      25.676583               <NA>  ...               <NA>   

  OriginOID DestinationOID                                      From_Address  \
0         1              1  904 West Hillcrest Drive, Johnson City, TN 37604   
1         1              2  904 West Hillcrest Drive, Johnson City, TN 37604   
2         1              3  904 West Hillcrest Drive, Johnson City, TN 37604   

   From_Type             From_OBJ

In [47]:
od_df.to_csv("test.csv")

In [21]:
od_df2 = od_df[['From_OBJECTID','To_OBJECTID','Total_Miles','Total_Minutes']]
print(od_df2)

              From_OBJECTID               To_OBJECTID  Total_Miles  \
0  65d75a2ce09e67e025e8cd30  65d75c87e09e67e025e8cd33     3.290089   
1  65d75a2ce09e67e025e8cd30  65d75cb9e09e67e025e8cd34    13.811811   
2  65d75a2ce09e67e025e8cd30  65d75ce1e09e67e025e8cd35    19.906688   

   Total_Minutes  
0       7.917875  
1      23.989688  
2      25.676583  
